# Bibliotecas

In [36]:
import pandas as pd

# Limpeza dos dados

## Lendo os arquivos

In [37]:
df_americanas = pd.read_csv(f"Dados/produtos_americanas.csv", dtype = {'ean': str})
df_amazon = pd.read_csv(f"Dados/produtos_amazon.csv", dtype = {'ean': str})

In [38]:
df_amazon["ean"].value_counts()

0713477350430    57
9095247871972    51
7908414420304     2
7319625661149     1
7465214678507     1
                 ..
0708570152586     1
8263430934112     1
6973553520094     1
3041412361256     1
6934177770012     1
Name: ean, Length: 1017, dtype: int64

In [91]:
df_amazon_ft[df_amazon_ft["ean"].notnull()].shape

(636, 5)

## Retirando títulos com palavras indesejadas

In [102]:
bad_words = ["alargadores", "anel", "atomiser", "biometria", "box",
             "cable", "cabo", "caixa", "campainha", "capa", "capinha", "card", "case", "caso", "carregador",
             "chassi", "chaveiro", "chave seletora", "clip", "collar", "comoda", "cômoda", "cordão", "cortador",
             "drone", "eon", "eixo", "eixos", "espelho", "filme", "franquia", "gancho", "grânulos",
             "handmade", "hemobblo", "hemobllo", "jbl", "kit",
             "lente", "lixa", "lurrose", "m-disc", "microsd", "monitor", "moochy", "netcna",
             "papel", "parede", "pneu", "pneus", "power bank", "prettyzoom", "pretyzoom", "pulseira",
             "radiador", "realidade virtual", "refrigerador", "reparo", "reposição", "reposiçao",
             "resfriador", "retrovisor", "ronyme", "rv",
             "streaming box", "smart tv", "soquete", "suporte",
             "tampa", "tomada", "tomadas", "tradutor", "turbopower",
             "verbatim", "vidro", "vr", "zbox", "watch"
            ]

df_amazon['titulo_cb'] = df_amazon['titulo'].str.lower()
df_americanas['titulo_cb'] = df_americanas['titulo'].str.lower()

df_amazon_ft = df_amazon[~df_amazon["titulo_cb"].str.contains('|'.join(bad_words))]
df_americanas_ft = df_americanas[~df_americanas["titulo_cb"].str.contains('|'.join(bad_words))]
print(f"Amazon: {df_amazon_ft.shape}\nAmericanas: {df_americanas_ft.shape}")

Amazon: (2210, 5)
Americanas: (899, 5)


In [103]:
print(f"Amazon: {df_amazon.shape}\nAmericanas: {df_americanas.shape}")

Amazon: (4333, 5)
Americanas: (1032, 5)


In [101]:
df_amazon_ft['url'].loc[211]

'https://www.amazon.com.br/Lurrose-Adaptador-Expans%C3%A3o-Acess%C3%B3rio-Computador/dp/B09H431X9H/ref=sr_1_2495?qid=1651784177&s=wireless&sr=1-2495&ufe=app_do%3Aamzn1.fos.95de73c3-5dda-43a7-bd1f-63af03b14751'

In [104]:
df_amazon_ft[50:].head(50)

,url,titulo,ean,descricao,titulo_cb
133,https://www.amazon.com.br/Smartphone-Samsung-B...,Smartphone Samsung Galaxy S21 5G 128gb 8gb Ram...,NaN,Sobre este item [Smartphone Samsung Galaxy ...,smartphone samsung galaxy s21 5g 128gb 8gb ram...
135,https://www.amazon.com.br/Xiaomi-Daybreak-Purp...,Redmi Note 9t 5g Xiaomi 4gb 64gb Daybreak Purp...,NaN,Sobre este item Náo funciona no Verizon Boo...,redmi note 9t 5g xiaomi 4gb 64gb daybreak purp...
136,https://www.amazon.com.br/Smartphone-5000mAh-W...,"Smartphone C23 Pro, 4GB + 64GB, 5000mAh, 4G, t...",NaN,Sobre este item Sistema operacional: Androi...,"smartphone c23 pro, 4gb + 64gb, 5000mah, 4g, t..."
138,https://www.amazon.com.br/Smartphone-ASUS-ZS67...,Smartphone ASUS Zenfone 8 Flip ZS672KS-2A030BR...,NaN,Sobre este item 1.Câmera Flip tripla trasei...,smartphone asus zenfone 8 flip zs672ks-2a030br...
139,https://www.amazon.com.br/Smartphone-forte-dur...,"Smartphone S59, forte e durável, 10050mAh, 4GB...",NaN,Sobre este item Cartão SIM: dois cartões SI...,"smartphone s59, forte e durável, 10050mah, 4gb..."
141,https://www.amazon.com.br/Xiaomi-Redmi-Note-Gr...,Xiaomi Redmi Note 11S 6GB 128GB Graphite Gray ...,NaN,NaN,xiaomi redmi note 11s 6gb 128gb graphite gray ...
142,https://www.amazon.com.br/XIAOMI-10T-ATLANTIC-...,XIAOMI MI 10T LITE ATLANTIC BLUE 128GB 6GB RAM...,NaN,NaN,xiaomi mi 10t lite atlantic blue 128gb 6gb ram...
146,https://www.amazon.com.br/Samsung-Smartphone-G...,Samsung Smartphone Galaxy S9 256gb 4gb Ram Tel...,NaN,Sobre este item Samsung Smartphone Galaxy S...,samsung smartphone galaxy s9 256gb 4gb ram tel...
147,https://www.amazon.com.br/Smartphone-Xiaomi-Mi...,Smartphone Xiaomi Mi11 Lite 5G NE peach Pink 8...,NaN,NaN,smartphone xiaomi mi11 lite 5g ne peach pink 8...
149,https://www.amazon.com.br/Ultra-Smartphone-Fin...,S22 Ultra 5G Smartphone Fingerprint Recognitio...,NaN,"Sobre este item Tela cheia de 6,93 polegada...",s22 ultra 5g smartphone fingerprint recognitio...


In [45]:
ean_amz = list(set(df_amazon["ean"].to_list()))[1:]

#match_ean = df[df["ean"].isin(ean_amz)]["ean"]

In [46]:
df_amazon.loc[~df_amazon["ean"].duplicated(keep='first')]["ean"].value_counts()

0774826534659    1
8434387544314    1
6941059643524    1
6901905889520    1
6912690582886    1
                ..
7892597351640    1
0708570152586    1
8263430934112    1
6973553520094    1
6934177770012    1
Name: ean, Length: 1017, dtype: int64

In [47]:
df1 = df_amazon.loc[~df_amazon["ean"].duplicated(keep='first')]
df2 = df_americanas.loc[~df_americanas["ean"].duplicated(keep='first')]

pd.concat([df1.set_index('ean'),df2.set_index('ean')], axis=1, join='inner')

,url,titulo,descricao,titulo_cb,descricao,titulo,url,titulo_cb
ean,,,,,,,,
NaN,https://www.amazon.com.br/Ulefone-desbloqueio-...,Ulefone NOTE 10 Android 11 desbloqueio de smar...,Sobre este item ★ 【SO Android 11 Go Edition...,ulefone note 10 android 11 desbloqueio de smar...,"Smartphone Multilaser F, trazendo mais versati...",martphone Multilaser F 3G 16GB 1GB Tela 5.5 Se...,https://www.americanas.com.br/produto/1517503241,martphone multilaser f 3g 16gb 1gb tela 5.5 se...
7892509121330,https://www.amazon.com.br/Smartphone-Samsung-G...,Smartphone Samsung Galaxy A52s Rede 5G 128Gb 6...,NaN,smartphone samsung galaxy a52s rede 5g 128gb 6...,Celular smartphone samsung galaxy A52S 5G duos...,Celular Samsung Galaxy A52s 5G Preto 128GB Tel...,https://www.americanas.com.br/produto/4888541757,celular samsung galaxy a52s 5g preto 128gb tel...
7892597351220,https://www.amazon.com.br/Smartphone-Motorola-...,Smartphone Motorola Edge 20 Pro 256GB 12GB RAM...,Sobre este item Abuse de imagens fenomenais...,smartphone motorola edge 20 pro 256gb 12gb ram...,Todas as informações divulgadas são de respons...,Smartphone Motorola Edge 20 Pro 256GB 5G Wi-Fi...,https://www.americanas.com.br/produto/3616548576,smartphone motorola edge 20 pro 256gb 5g wi-fi...
6941399013094,https://www.amazon.com.br/RealMe-RMX2001-poleg...,"RealMe 6 128 GB + 4 GB RMX2001 6,5 polegadas D...",Sobre este item Versão GSM (não funciona co...,"realme 6 128 gb + 4 gb rmx2001 6,5 polegadas d...",Smartphone Realme 6 Dual Sim 4gb Ram 128gb Azu...,Smartphone Realme 6 Dual Sim 4gb Ram 128gb Azul,https://www.americanas.com.br/produto/4370722799,smartphone realme 6 dual sim 4gb ram 128gb azul
7892509117722,https://www.amazon.com.br/Smartphone-Samsung-G...,Smartphone Samsung Galaxy A32 128GB 6.4 Octa C...,NaN,smartphone samsung galaxy a32 128gb 6.4 octa c...,O Samsung Galaxy A32 tem tudo que você quer! C...,Celular Samsung Galaxy A32 128GB Octa Core Câm...,https://www.americanas.com.br/produto/3835784886,celular samsung galaxy a32 128gb octa core câm...
7892597350315,https://www.amazon.com.br/Celular-Motorola-Mot...,Smartphone Motorola Moto E7 32GB 2GB RAM Cinza...,Sobre este item Câmera dupla com sensor de ...,smartphone motorola moto e7 32gb 2gb ram cinza...,Você merece ter um smartphone tecnológico e id...,Smartphone Motorola Moto E7 32GB 4G Wi-Fi Tela...,https://www.americanas.com.br/produto/2626417844,smartphone motorola moto e7 32gb 4g wi-fi tela...
7908414420304,https://www.amazon.com.br/Smartphone-Multilase...,Smartphone Multilaser G Max 2 2GB + 64GB Tela ...,Sobre este item Câmera Tripla 8MP + VGA + V...,smartphone multilaser g max 2 2gb + 64gb tela ...,Multilaser G Max 2 com tela de 6.5 hd + u-notc...,Smartphone GMax 2 P9157 Tela 6.5 64GB 2GB ram ...,https://www.americanas.com.br/produto/4782017527,smartphone gmax 2 p9157 tela 6.5 64gb 2gb ram ...
7892509117265,https://www.amazon.com.br/Smartphone-Samsung-O...,Smartphone Samsung Galaxy A12 Dual Chip Androi...,Sobre este item Cor: Preto Dual: Sim ...,smartphone samsung galaxy a12 dual chip androi...,O Smartphone Samsung Galaxy A12 conta com desi...,Smartphone Samsung Galaxy A12 64GB 4G Wi-Fi Te...,https://www.americanas.com.br/produto/2655733694,smartphone samsung galaxy a12 64gb 4g wi-fi te...
7892597349692,https://www.amazon.com.br/Smartphone-Moto-Edge...,"Smartphone Moto Edge+ Thunder Grey 6,7"" 5G 256...",NaN,"smartphone moto edge+ thunder grey 6,7"" 5g 256...",Todas as informações divulgadas são de respons...,Smartphone Motorola Edge+ 256GB 5G Wi-Fi Tela ...,https://www.americanas.com.br/produto/1778939676,smartphone motorola edge+ 256gb 5g wi-fi tela ...


## Fazendo match